# 6. Labor: Többosztályos klasszifikáció Neurális Hálózattal

A gyakorlat során az egyszerű MLP neurális hálót fogjuk adaptálni kettőnél több osztályba való kalsszifikációhoz.

### Kézzel írott számjegyek felismerése

A több osztályba történő klasszifikáció feladatát a kézzel írott számjegyeket tartalmazó MNIST dataset egy származtatásán végezzük el. A cél egy olyan egyszerű MLP modell felépítése, amely képes egy 20x20-as szürkeárnyalatos kép alapján meghatározni milyen számjegy (0-9) szerepel az adott mintán.

## Többosztályos klasszifikáció

Az eddigiekben a bináris klasszifikáció feladatára láttunk példákat a logisztikus regresszió / perceptron struktúra alkalmazásával. Hogyan lehetne a perceptron struktúra felhasználásával egy olyan modellt készíteni, ami a bemenetek alapján képes 10 különböző osztály közül választani?
- Perceptron 0-9 közötti kimenettel?

Ha 10 osztály közül kell választani, akkor egy [0,10] kimeneti tartományú modell megfelelőnek tűnhet, azonban jelentősen korlátozza a megoldás lehetőségeit a kimenetek közötti hierarchia feltételezésével: *a '4' számjegyet tartazó kép bemeneti alapján a kimenet kétszer akkora legyen, mintha a kép egy '2' számjegyet tartalmazna*; *a macskát ábrázoló kép (3. osztály) a kimenetre harmadakkora értéket eredményezzen, mintha a képe nyulat (9. osztály) ábrázolna*. Könnyen belátható, hogy ez nem egy jó megközelítés, ritákn célszerű a mintákra ilyen hierarchiát rákényszeríteni.

- Külön-külön modell minden osztályba való tartozásra

Egy perceptronnal egy bináris klasszifkációs probléma modellezhető. Az eredeti feladatot (*melyik állat szerepel a képen?*) felbonthatjuk több, egyenként bináris kalsszifikációt tartalmazó feladatra (*mekkora az esélye, hogy macskát ábrázol a kép?*, *mekkora az esélye, hogy nyulat a kép?*, *stb*). Az egyes perceptronokkal egy **One vs. All** klasszifikációs feladatot valósítunk meg (*mekkora a valószínűsége, hogy az adott osztályba és nem bármelyik másikba tartozik inkább a bemeneti adatpont?*)

10 osztály esetén ez 10 különböző logisztikus regresszió illesztése, amelyek egyenként az adott osztályra ad becslést a **One vs. All** struktúrának megfelelően. A gyakorlaton használt adatokra 10 független logisztikus regresszió illesztésével a kézzel írott számjegyek felismerésének feladata (több-kevesebb pontossággal) megoldható a **One vs. All** struktúrát alkalmazva.

<!---
<center><img src="img/oneVSall.svg" width="400"></center>
-->
<center><img src="https://drive.google.com/uc?export=view&id=1I1TZHLYuVGekQFuD-M-343i5NXKjHyTx" width="400"></center>



Vegyük észre, hogy az azonos bemeneti adatokon 10 különböző bináris klasszifikációt megvalósító perceptronok képe megyegyezik egy MLP modell 10 neuront tartalmazó rétegének alakjával. Amennyiben ez a réteg az MLP modell utolsó (kimeneti) rétege, úgy egy olyan neurális háló struktúra adódik, amely a bemenet 10 különböző osztályba való tartozásnak a valószínűségét képes modellezni. Ekkor elmondható, hogy a rejtett rétegek a bemenetek magasabb absztrakciós szintű *'tulajdonságait'* határozzák meg, amely jól reprezentálja a bemenetet, majd a legutolsó, kimeneti réteg ezen absztrakt tulajdonságok alapján végzi el az adott osztályba való tartozás valószínűségének megbecslését. Állatok osztályozásánál például a bemeneti kép alapján elképzelhető, hogy az utolsó rejtett rétegben egy neuron a szőrösséget, egy másik kimenet a lábak számát, míg megint egy másik kimenet a testméretet, vagy a testszínt írja le. A klasszifikáció ezen tulajdonságok alapján történik az eredeti egy-egy pixel adatot tartalmazó változók helyett.

## One-hot encoding

A bemenetekhez tartozó címkék a legtöbb esetben a bemenethez tartozó osztály számaként, vagy az osztálynak megfelelő szöveges formátumú címkeként vannak megadva (sparse encoding).

$$ y_l = 3; \qquad y_l = \text{"cat"} $$

A fenti modell esetében egy bemenetre a kimenet az osztályok számának megfelelő darab valószínűségi értéket fog tartalmazni egy sorvektor formájában. A modell tanításához a bemeneti adatokhoz tartozó cimkéket ennek a struktúrának megfelelően kell megadni. Amennyiben az osztályok száma $c$ :

$$\mathbf{y}_l = 
	\left[ \begin{array}{cccc}
 		y_{l, 1} & y_{l, 2} & \cdots & y_{l, c}\\
	\end{array}\right]
$$

Mivel a bemenet pontosan egy osztályba tartozik, a $y_{l, j}$ értékek közül pontosan egy lesz 1, a többi 0. Egy 4 különöző osztályt megkülönböztető példa esetén, ahol a bemenet a harmadik osztályba tartozik:

$$\mathbf{y_l} = 
	\left[ \begin{array}{cccc}
 		0 & 0 & 1 & 0\\
	\end{array}\right]
$$

Ezt a formátumot 'one-hot' enkódolt formátumnak nevezik, mivel a lehetséges értékek közül pontosan egy érvényes, vagy 'hot'. A feladatmegoldás során a kapott címkéket ilyen formátumra kell konvertálnunk. Ennek implementációját segíti, hogy számmal adott osztálycímkék esetén a one-hot enkódolt érték pont megfelel egy $c$ méretű egységmátrix megfelelő sorának:

$$\mathbf{Y}_{sparse} = \left[ \begin{array}{c}
 		 1 \\
		 2 \\
		 3 \\
		 4 \\
	\end{array}\right] \Rightarrow

\mathbf{Y}_{one-hot} = \left[ \begin{array}{cccc}
 		 1 & 0 & 0 & 0\\
		 0 & 1 & 0 & 0\\
		 0 & 0 & 1 & 0\\
		 0 & 0 & 0 & 1\\
	\end{array}\right]$$

Szövegként megadott címkék esetén hasonlóan járhatunk el, ha egy listába rendezzük a különböző címkéket, majd az adott osztály listában elfoglalt helye szerint rendeljük hozzá a one-hot enkódolt kimenetet.

## SoftMax aktivációs függvény

A kimeneti rétegben *sigmoid* aktivációs függvényt alkalmazva a modell minden osztályba való tartozásra meghatároz egy valószínűséget. A single label osztályozás feladattípusnál (egy bemenet pontosan egy osztályba tartozik) az összes osztályra vett valószínűségek összegének egyet kell adnia, mivel annak a valószínűsége, hogy a bemenet az adott osztályok közül valamelyikbe tartozik pontosan egy (biztos esemény). Matematikailag azonban semmi nem akadályozza, hogy a modell egy adott bemenetre olyan valószínűségeket prediktáljon, melyek összege nem egy. Például állatokról készült képek klasszifikációjánál előfordulhat, hogy a predikció szerint 0.75 eséllyel kutya és 0.80 eséllyel macska. 

A kimeneti rétegre ezért egy más aktivációs függvényt kell alkalmaznunk, amely a kimeneti réteg $\mathbf{s}_l^{(p)}$ értékei alapján úgy határozza meg a kimeneteket, hogy azok tartománya [0,1] legyen, összegük 1 és értékeik az $\mathbf{s}_l^{(p)}$ értékekkel nem feltétlenül lineárisan, de arányosak (nagyobb $s$ értékhez nagyobb kimeneti valószínűség tartozzon). Jó megoldásnak tűnhet erre, ha a az adott kimenethez tartozó $s^{(p)}_{l,j_0}$ értéket az adatponthoz tartozó összes $s^{(p)}_{l,j}$ érték összegével normáljuk:

$$\hat y_{l,j_0} = a(s^{(p)}_{l,j_0}) = \frac{s^{(p)}_{l,j_0}}{\sum_{j=1}^c s^{(p)}_{l,j}}$$

Amíg minden $s^{(p)}_{l,j}$ érték pozitív, ez teljesíti a követelményeket, miszerint

$$ \sum_{j=1}^c \hat y_{l,j_0} = 1$$
és
$$ a(s^{(p)}_{l,j_a}) > a(s^{(p)}_{l,j_b}) \quad \text{ha} \quad s^{(p)}_{l,j_a} > s^{(p)}_{l,j_b}$$

Könnyen látható azonban, hogy ha az $s^{(p)}_{l,j}$ értékek között szerepel negatív is, a feltételek sérülnek. érdemes tehát először transzformálnunk ezeket az összegeket egy olyan függvénnyel, amely a $[-\infty, +\infty]$ tartományról a $[0, +\infty]$ tartományra képez, és szigorúan monoton növekvő függvény. Ezeknek a feltételeknek eleget tesz az exponenciális függvény, amellyel az új aktivációs függvény:

$$ a(s^{(p)}_{l,j_0}) = \frac{e^{s^{(p)}_{l,j_0}}}{\sum_{j=1}^c e^{s^{(p)}_{l,j}}}$$

Az így kapott *softmax* aktivációs függvény (amelyet kizárólag a kimeneti rétegben szokás alkalmazni többosztályos klasszifikáció esetén) teljesíti mind a valószínűségek összegére, mind a bemenetekkel való arányosságra vonatkozó követelményeket:

$$ \boxed{softmax(z_{j_0}) = \frac{e^{z_{j_0}}}{\sum_{j=1}^c e^{z_j}}}$$

## Kategórikus keresztentrópia

A több kategóriás klasszifikációhoz a bináris klasszifikáció - BCE költséghez hasonlóan  a *Maximum Likelihood* becslést alklamazva rendelhető költségfüggvény. A modell paramétereitől függően az összes $m$ adatpontra a likelihood függvény $\mathcal{L}(\mathbf{w} \ | \ \mathbf{X})$ az egyes adatpontok valószínűségének szorzata:

$$ \mathcal{L}(\mathbf{w} \ | \ \mathbf{X}) = \prod_{l=1}^m  \mathcal{L}(\mathbf{w} \ | \ \mathbf{x}_{l}) $$

ahol $\mathbf{w}$ itt a modell *összes* paraméterét jelöli, $\mathbf{X}$ az összes bemeneti adatpont minden változóját tartalmazó mátrix, míg $\mathbf{X}_{l}$ az $l$-edik adatpont változóit tartalmazó (sor)vektor.
Egy bemeneti adatpont előfordulásának valószínűsége a $\mathbf{w}$ modellparaméterek mellett a bementek tényleges osztályára prediktált valószínűség:

$$ \mathcal{L}(\mathbf{w} \ | \ \mathbf{x}_{l}) = \hat y_{l,j_{l}} $$

ahol $j_{l}$ az $l$-edik adatpont valós osztálya.

A bináris keresztentrópiához hasonlóan a likelihood helyett annak logaritmusának, a log-likelihood függvény alakja:

$$ ln \left(\mathcal{L}(\mathbf{w} \ | \ \mathbf{X})\right) = ln \left( \prod_{l=1}^m  \hat y_{l,j_{l}} \right) $$

A $\log(a \cdot b) = \log(a)+\log(b)$ azonosság alapján:

$$ ln \left(\mathcal{L}(\mathbf{w} \ | \ \mathbf{X})\right) =  \sum_{l=1}^m ln \left( \hat y_{l,j_{l}} \right) $$

A $\hat y_{l,j_{l}}$ specifikus kiválasztása helyett a one-hot enkódolt kimeneti vektorokat felhasználva

$$ln \left(\hat y_{l,j_{l}} \right) = \sum_{j=1}^c y_{l,j} \ ln \left(\hat y_{l,j} \right)$$  

ahol $c$ a feladathoz tartozó osztályok száma. A teljes log-likelihood függvény így:

$$ ln \left(\mathcal{L}(\mathbf{w} \ | \ \mathbf{X})\right) =  \sum_{l=1}^m \sum_{j=1}^c \left( y_{l,j} \ ln \left(\hat y_{l,j} \right) \right) $$

A log-likelihood függvény maximalizálása megegyezik a a függvény negatívjának minimalizálsával, így a kategórikus keresztentrópia (Categorical Cross-Entropy) költségfüggvény értéke, a bemenetek számától függetlenítve:

$$ \boxed{C_{CCE} = \frac{1}{m} \sum_{l=1}^m \sum_{j=1}^c \left( - y_{l,j} \ ln \left(\hat y_{l,j} \right) \right) }$$

## BackProp a többosztályos klasszifikáció esetén

A többosztályos klasszifikáció esetén a kimeneti réteg $\mathbf{\delta}^{(p)}$ értékének számítását az alkalmazott CCE költségfüggvény és a *softmax* aktivációs függvény apaján kell elvégezni. Mivel az egyes $\mathbf{\hat y}_{l,j}$ kimenetek az utolsó réteg minden $\mathbf{S}_l$ értékétől függenek a *softmax* miatt, a számítása nem triviális. A számítások elvégzése után az eddigiekkel megegyező formát kapunk. A teljes levezetés az **Appendix** részben megtalálható.

$$ \mathbf{\delta}^{(p)} = \frac{1}{m} \left(\mathbf{\hat Y} - \mathbf{Y}\right) $$

A többosztályos klasszifikációnál használt hálózat rejtett rétegeinek struktúrája az eddigiekhez képest változatlan. Az itt használható szokásos aktivációs függvények ritkán tartalmaznak a réteg többi $s$ értékétől való függést, így a backprop folyamata ezekre a rétegekre változatlan:

$$ \mathbf{\delta^{(k)}} = \left( \mathbf \delta^{(k+1)} \cdot \left( \mathbf W^{(k+1)}_{-BIAS} \right)^T \right) a'\left(\mathbf s^{(k)} \right) $$

Illetve a gradiensek értéke minden rétegben az eddigiekkel megegyezően:

$$ \frac{\partial C}{\partial \mathbf{W}^{(k)}} = \left(\mathbf X^{(k)}\right)^T \cdot \mathbf \delta^{(k)} $$

## 00: Könyvtár importálások

Első lépésként importáljuk a feladat megoldása során használt könyvtárakat. Esetünkben ezek a következők lesznek:
- Numpy a matematikia műveletek elvégzéséhez
- Pandas az adatok beolvasásához és kezeléséhez
- MatPlotLib.pyplot az eredményeink ábrázolásához
- SciPy 'loadmat' modulja a matlab változókat tartalamzó fájlok importálására
- SciKit-Learn confusion_matrix számoló modulja a hálózat értékeléséhez

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix

# Használjuk ezeket sötét téma esetén
plt.style.use('dark_background')
styleTemplate = 'plotly_dark'

# Használjuk ezeket világos téma esetén
#plt.style.use('default')
#styleTemplate = 'plotly_white'

## 01: Adatbeolvasás
Az adatok az  `mnist.mat` matlab változófájlban találhatók. Ilyen fájltípus a SciPy `loadmat` moduljával importálhatók.

In [ ]:
mat = loadmat("mnist.mat")                 # .mat file importálás
X = mat["X"]                               # X rendezése
Y = mat["y"]                               # Y rebbdezése
m = X.shape[0]                             # minták száma
n = X.shape[1]                             # változók száma
c = np.unique(Y).size                      # osztályok száma

print('''Shape of the dataset in order X and Y:
''',X.shape,'\n',Y.shape,'\n')

## 02: Adatfelfedezés

A beolvasott adatainkon látszik, hogy 5000 mintát tartalmaz. A bemenet 400 változót tartalmaz, amely a 20x20-as kép egyes pixelei egy kiterített vektorban, az alábbihoz hasonló logika szerint (a jelen adataink sorok helyett oszloponként vannak kiterítve - *Fortran-like index order*). 
<!--
<center><img src="img/img_flatten.svg" width="455"></center>
-->
<center><img src="https://drive.google.com/uc?export=view&id=1HjqjYJVmAouaV2nD-SQplYCBeaNAPybP" width="455"></center>

A modellválasztás nem feltétlenül optimális, mivel a Feed forward neurális háló bemenetei között nincs struktúra feltételezve, véletlenszerű pixlesorrendben ugyan annyi infromációt tudnánk átadni. A modellünk a síkbeli struktúrát nem fogja jól érzékelni (melyik pixel melyik felett/mellett/alatt van), azonban a feladat egyszerűsége miatt még így is eredményes. Magasabb komplexitás esetén érdemes konvolúciós neurális háló struktúrát alkalmazni kép jellegű bemenetekre, amely képes az egyes pixelek egymáshoz képesti elhelyezkedését is figyelembe venni.

Ábrázoljuk ez első 10 mintát, hogy jobb képet kapjunk az adatainkról, illetve vizsgáljuk meg a bemeneti adataink nagyságrendjét! Először nézzük meg a címkék formátumát.

In [ ]:
print("Az adatokban található osztályok: {}".format(np.unique(Y)))

print("Az első 10 mintához tartozó címkék: {}".format(Y[:10,:].flatten())) # kiíratás sorvektorként

Ez alapján megállíptható, hogy 10 különböző osztály áll rendelkezésünkre (0-9 számjegyek), láthatóan rendezett sorrendben. Ábrázoljunk egy pár véletlen mintát, a hozzájuk tartozó címkékkel.

In [ ]:
print("15 véletlenszerű minta ...")                                    # kirajzoltatjuk az első 100 elemet
rng = np.random.default_rng(42)                                   # véletlenszám generátor
samples = rng.integers(m, size=15)
fig, axis = plt.subplots(1,15, figsize = (10,4))                # mivel egy plotra akarunk több ábrát -> subplots
for i in range(15):                                             # 15 kis ábra
    pixels = X[samples[i],:].reshape([20,20], order='F')        # 20x20 alak visszaállítása, Fortran index order
    axis[i].imshow(pixels)                                      # kirajzolás
    axis[i].axis("off")
    axis[i].set_title(Y[samples[i],0])
plt.show()

print("A bemeneti adatok (pixelértékek) tartománya: [{}, {}]".format(np.min(X), np.max(X)))

Az ábrázolás alapján látszik, hogy (feltehetően a matlabos indexelés miatt) a 0-ás számjegyhez 10-es címke van rendelve. A bemeneti adatok bár nem a megszokott 0..1 vagy 0..255 tartományon vannak, nagyságrendük nem igényli a normalizálást.

**Megjegyzés:** a képek szürkeárnyalatos formában adottak (1 érték pixelenként), a színskálát a matplotlib rendeli hozzá.

## 03: Adatok előkészítése

A legtöbb esetben nem szerencsés a tanításra használt adatokat rendezett módon megadni (ennek jelentősége akkor nagyobb, ha a súlymódosítást nem a teljes adathalmazra számoljuk, hanem 'batch' módban, egy-egy kissebb részre külön-külön végezzük el). Az adatok előkészítése így jelen esestben a követtkező lépésekből áll:
- adatok sorrendjének randomizálása
- 10-es címkék javítása 0-ra
- one-hot enkódolt kimenetek előállítása

In [ ]:
# Shuffle data
shuffledData = np.hstack([X,Y])
rng.shuffle(shuffledData, axis = 0)
X = shuffledData[:,:400]
Y = shuffledData[:,400:].astype('uint8')

# 10->0
Y[Y[:,0]==10, 0] = 0

# Méretek és keverés sikerességének ellenőrzése
print('''Shape of the dataset in order X and Y:
''',X.shape,'\n',Y.shape,'\n')
print("Az első 20 mintához tartozó címke: {}".format(Y[:20,:].flatten())) # kiíratás sorvektorként

Láthatóan sikeres volt az elemek keverése és a 0 számjegyekhez tartozó címkék cseréje.

**Feladat:** állítsa elő a címkéknek megfelelő one-hot enkódolt kimeneti mátrixot!

In [ ]:
######################################################


######################################################

print("Az első 6 mintához várt one-hot enkódolt címkék:")
print('''[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]''')
print("Az első 6 mintához tartozó tényleges one-hot enkódolt címkék:")
print(Y_oneHot[:6,:])

## 04: Modell implementálása

Az előző órán elkészített egyszerű neurális háló modellből kiindulva végezzük el a szükséges módosításokat a többosztályos klasszifikáció implementálásához.

In [ ]:
def sigmoid(z, derivate = False):       # Alapértelmezett a rendes sigmoid érték számítása
    g = 1 / (1 + np.exp(-z))
    if derivate == True:
        g = g * (1-g)
    return g

**Feladat:** implementálja a `softmax()` aktivációs függvényt, amely a bemenetként adott $\mathbf{S}^{(p)}$ mátrixra kiszámítja a kimeneti $\mathbf{\hat Y}^{(p)}$ mátrixot a *softmax* aktivációnak megfelelően!

In [ ]:
def softmax(z):       # Alapértelmezett a rendes sigmoid érték számítása
######################################    



######################################
    return g

# Test softmax
z = np.array([[2.52640812, 4.75720026, 5.76544649],
       [4.93992941, 6.70753489, 4.90961306],
       [5.64891358, 5.49916996, 4.72750292],
       [5.26816593, 6.27247156, 2.79979483]])

print()
print('''Expected softmax outputs for test array:
[[0.02792016 0.2598595  0.71222034]
 [0.12776367 0.74828786 0.12394847]
 [0.4426961  0.38112977 0.17617413]
 [0.26214149 0.71564916 0.02220935]]''')
print('Actual softmax outputs for test array:')
print(softmax(z))

In [ ]:
def addBias(z):
    return np.hstack([np.ones([z.shape[0] ,1]), z])

**Feladat:** végezze el a szükséges változtatásokat az alábbi bináris klasszifikációra implementált neurális háló osztályon, hogy az képes legyen a többosztályos klasszifikáció elvégzésére!

In [ ]:
class FeedForwardNN:
    def __init__(self, layerSizes, activationFunction, seed):
        self.layerSizes = layerSizes            # Háló konfigurációja
        self.a = activationFunction             # Aktivációs függvény (később más aktivációs függvény is alkalmazható)
        self.noLayers = len(layerSizes)-1       # Rétegek száma (iterációhoz, bemeneti réteg nélkül)
        self.inputSize = layerSizes[0]          # Bemeneti változók (input feature) száma
        self.Yhat = []                          # Lista az egyes rétegek kimenetének tárolására
        self.X = []                             # Lista az egyes rétegek bemenetének tárolására
        self.W = []                             # Lista a rétegekhez tartozó súlyoknak
        self.initWeights(seed)                  # Súlyok inicializálása
    
    def initWeights(self, seed = None):
        rng = np.random.default_rng(seed)
        # Weight matrix dimension are based on number of neurons in previous and current layer
        self.W = [rng.uniform(low = -1, high = 1, size = [self.layerSizes[i]+1, self.layerSizes[i+1]]) for i in range(self.noLayers)]

    def checkInputSize(self, X):
        if X.shape[1] != self.inputSize:
            raise ValueError('Unexpected number of input features! Expected {} features but got {}.'.format(self.inputSize, X.shape[1])) 

    def forwardProp(self, X):
        self.checkInputSize(X)

        self.X = [[] for i in range(self.noLayers)]
        self.Yhat = [[] for i in range(self.noLayers)]
        
        self.X[0] = addBias(X)  # FirstLayer
        self.Yhat[0] = self.a(self.X[0]@self.W[0])

        for i in range(1, self.noLayers-1): # HiddenLayers
            self.X[i] = addBias(self.Yhat[i-1])
            self.Yhat[i] = self.a(self.X[i]@self.W[i])

        self.X[-1] = addBias(self.Yhat[-2])
        self.Yhat[-1] = sigmoid(self.X[-1]@self.W[-1])

    def predict(self, X):
        self.forwardProp(X)
        return self.Yhat[-1]

    def costBCE(self, X, Y, forwardProp = True):
        eps = 10e-15
        if forwardProp: self.forwardProp(X)
        return np.mean(-Y*np.log(self.Yhat[-1]+eps)-(1-Y)*np.log(1-self.Yhat[-1]+eps))
    
    def accuracy(self, X, Y, forwardProp = True):
        if forwardProp: self.forwardProp(X)
        return np.mean(Y == (self.Yhat[-1] > 0.5)) * 100

    def backProp(self, trueY):
        deltas = [[] for i in range(self.noLayers)]
        m = trueY.shape[0]

        # Delta a kimeneti rétegben
        deltas[-1] = 1/m * (self.Yhat[-1] - trueY)
        # Delta kiszámítása a rejtett réteg(ek)ben
        for i in range(self.noLayers-2, -1, -1):
            deltas[i] = (deltas[i+1]@self.W[i+1][1:,:].T) * self.a(self.X[i]@self.W[i], derivate = True)
        
        dW = [self.X[i].T@deltas[i] for i in range(self.noLayers)]
        return dW

    def updateWeights(self, learning_rate, dW):
        for i in range(self.noLayers):
            self.W[i] = self.W[i] - learning_rate * dW[i]

    def fit(self, X, Y, learning_rate, epochs):
        self.forwardProp(X)

        C_history = np.zeros([epochs+1])
        C_history[0] = self.costBCE(X, Y, forwardProp = False)

        acc_history = np.zeros([epochs+1])
        acc_history[0] = self.accuracy(X, Y, forwardProp = False) 
        print('''
        \%\%\% ------- TANÍTÁS ------- \%\%\%
        ''')
        for i in range(epochs):
            dW = self.backProp(Y)
            self.updateWeights(learning_rate, dW)
            self.forwardProp(X)                     # Forwardprop with new weights
            C_history[i+1] = self.costBCE(X, Y, forwardProp = False)     
            acc_history[i+1] = self.accuracy(X, Y, forwardProp = False) 

            if ((i+1) % 100) == 0:
                
                print('Epoch {} / {} completed. Cost value: {}; Accuracy: {:.2f}'.format(i+1, epochs, C_history[i+1], acc_history[i+1])) 

        return C_history, acc_history

A modell helyes működése az alábbi tesztek futtatásával ellenőrizhető:

In [ ]:
layerSizes = [2, 3, 4]
seed = 42

testNN = FeedForwardNN(layerSizes, sigmoid, seed)

testX = np.array([[0.3146, -0.65432], [-1.0123, -0.4215], [0.2351, 0.7533456]])
testY = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0]])

In [ ]:
# Test initweights
print('''Expected initial weights with layer sizes {0} and random seed {1}:
[array([[ 0.5479121 , -0.12224312,  0.71719584],
       [ 0.39473606, -0.8116453 ,  0.9512447 ],
       [ 0.5222794 ,  0.57212861, -0.74377273]]), array([[-0.09922812, -0.25840395,  0.85352998,  0.28773024],
       [ 0.64552323, -0.1131716 , -0.54552256,  0.10916957],
       [-0.87236549,  0.65526234,  0.2633288 ,  0.51617548],
       [-0.29094806,  0.94139605,  0.78624224,  0.55676699]])]'''.format(layerSizes, seed))
print('''Actual initial weights with layer sizes {0} and random seed {1}:
{2}'''.format(layerSizes, seed, testNN.W))

In [ ]:
# Test forwardProp & Predict
testPred = testNN.predict(testX)

print('''Expected prediction for test parameters:
[[0.08834134 0.21661991 0.39782789 0.29721086]
 [0.07640996 0.21790532 0.39044155 0.31524317]
 [0.09595343 0.21828394 0.35748864 0.32827399]]'''.format(layerSizes, seed))
print('''Actual prediction for test parameters:
{0}'''.format(testPred))

In [ ]:
# Test backprop
testNN.forwardProp(testX)
testdW = testNN.backProp(testY)

print('''Expected weight change values for test parameters:
[array([[-0.03519045,  0.07277344,  0.04905192],
       [ 0.07725816, -0.10234505, -0.08827836],
       [ 0.00673892, -0.06410875, -0.04750743]]), array([[-0.24643176, -0.11573027,  0.04858602,  0.31357601],
       [-0.10767695, -0.11520305,  0.03385651,  0.18902349],
       [-0.16222644, -0.07032992,  0.07850704,  0.15404933],
       [-0.11605659, -0.05816385, -0.02614065,  0.20036109]])]''')

print('''Actual prediction for test parameters:
{0}'''.format(testdW))

## 05: Modell tanítása

Amennyiben minden tesztünkön megfelelő eredményt kaptunk, végezzük el a háló tanítását a `.fit()` metódus segítségével, és ábrázoljuk a költségfüggvény alakulását.

In [ ]:
learning_rate = 0.5
trainNN = FeedForwardNN([n, 25, c], sigmoid, 42)

C_history, acc_history = trainNN.fit(X, Y_oneHot, learning_rate, 400)

In [ ]:
fig, axis = plt.subplots(2,1)     

axis[0].plot(range(C_history.size), C_history)
axis[0].set_ylabel('Cost value [-]')

axis[1].plot(range(acc_history.size), acc_history)
axis[1].set_ylabel('Accuracy [%]')
axis[1].set_xlabel('epochs')
plt.show()

## 06: Modell értékelése

Jelen feladatnál már nehezen megoldható döntési határok vagy illesztett felületek vizsgálata. A modell értékelésének egy lehetséges módja egy véletlenszerű adatpont kiválasztása, és a modell kimenetének összevetése a valósággal.

In [ ]:
rndID = rng.integers(m)
pred = trainNN.predict(X[rndID, :].reshape(1,400))

plt.imshow(X[rndID, :].reshape([20,20], order='F'))
plt.show

print('Real label of data: {}'.format(Y[rndID,0]))
print('Predicted label of data: {} --> {}'.format(np.argmax(pred[0]), pred[0]))

A klasszifikáció értékelésénk egy másik lehetősége a konfúziós mátrix ábrázolása, amely megmutatja a valós és prediktált osztályok közötti kapcsolatokat.

In [ ]:
preds = trainNN.predict(X)
conf = confusion_matrix(Y, np.argmax(preds, axis = 1))  # Sparse predicitons

fig, ax = plt.subplots(figsize = (6,6))
im = ax.imshow(conf)

ax.set_xticks(np.arange(c), labels=np.unique(Y))
ax.set_xlabel('Prediktált számjegy')
ax.set_yticks(np.arange(c), labels=np.unique(Y))
ax.set_ylabel('Valós számjegy')

# Annotate cells
for i in range(c):
    for j in range(c):
        text = ax.text(j, i, conf[i, j],
                       ha="center", va="center", color="w")

ax.set_title("Kézzel írott számjegyek konfúziós mátrixa")
fig.tight_layout()
plt.show()

## 07: További tesztelés

Érdemes a modellt tovább vizsgálni, megfigyelni a tanítás paramétereinek hatásást, vagy akár implementálni a tanítóadatok szétbontását tanító és validációs halmazokra, és megvizsgálni a tanítás korai leállásának lehetőségét a validációs adatokon számított költségfüggvény alapján.

## XX: Megoldás Keras könyvtár segítségével

A bináris klasszifikációhoz hasonlóan ezt a feladatot is megodldhatjuk a amgaszintű Keras könyvtár segítségével.

In [ ]:
# imports for array-handling and plotting
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix

# keras imports for building and training our neural network
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

# Adatok előkészítése
mat = loadmat("mnist.mat")                 # .mat file importálás
X = mat["X"]                               # X rendezése
Y = mat["y"]                               # Y rendezése
c = np.unique(Y).size
Y[Y[:,0]==10, 0] = 0                       # 10 -> 0 label csere
oneHotMatrix = np.eye(c)
Y_oneHot = oneHotMatrix[Y[:,0],:]          # onehot Mátrix

# Keras háló definiálása
model = Sequential()
model.add(Dense(25, input_shape=(400,)))
model.add(Activation('sigmoid'))                            
model.add(Dense(10))
model.add(Activation('softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=SGD(learning_rate=0.5))

# training the model and saving metrics in history 
history = model.fit(X, Y_oneHot, epochs=10, verbose = 2) # Stochastic gradient descent can learn in a lot fewer epochs

# Plot metrics
fig, axis = plt.subplots(2,1)     

axis[0].plot(history.history['loss'])
axis[0].set_ylabel('Cost value [-]')

axis[1].plot(history.history['accuracy'])
axis[1].set_ylabel('Accuracy [%]')
axis[1].set_xlabel('epochs')
plt.show()

# Plot confusion matrix
preds = model.predict(X)
conf = confusion_matrix(Y, np.argmax(preds, axis = 1))

fig, ax = plt.subplots(figsize = (6,6))
im = ax.imshow(conf)

ax.set_xticks(np.arange(c), labels=np.unique(Y))
ax.set_xlabel('Prediktált számjegy')
ax.set_yticks(np.arange(c), labels=np.unique(Y))
ax.set_ylabel('Valós számjegy')

# Annotate cells
for i in range(c):
    for j in range(c):
        text = ax.text(j, i, conf[i, j],
                       ha="center", va="center", color="w")

ax.set_title("Kézzel írott számjegyek konfúziós mátrixa")
fig.tight_layout()
plt.show()

# Appendix
## BackProp a kimeneti rétegre multiclass classification feladat esetében

A backpropagation algoritmus implementálásához meg kell határoznunk a kimeneti réteghez tartozó $\mathbf{\delta}^{(p)}$ értékeket, amely alapján a háló többi $\mathbf{\delta}^{(k)}$ értéke számolható. Több kimeneti neuron esetében az egyes $\delta$ értékek meghatározása:

$$ \delta^{(p)}_{l,j} = \frac{\partial C}{\partial s^{(p)}_{l,j}}$$

**Megjegyzés**: a bináris klasszifikáció esetén ezt a kifejezést a lánc szabály segítségével tovább bonthatuk, mivel adatpontonként egyetlen $\hat y_{l}$ kimenet volt, amely kizárólag egy $s^{(p)}_{l}$ értéktől függött.

Multiclass classification feladat esetén a költségfüggvény a kategórikus keresztentrópia, az aktivációs függvény pedig a softmax függvény:

$$\frac{\partial C_{CCE}}{\partial s_{l_0,j_0}} = \frac{\partial \frac{1}{m} \sum_{l=1}^m \sum_{j=1}^c \left( - y_{l,j} \ ln \left(\hat y_{l,j} \right) \right)}{\partial s^{(p)}_{l_0,j_0}} = \frac{1}{m} \sum_{l=1}^m \sum_{j=1}^c \left( - y_{l,j} \frac{\partial ln \left(\hat y_{l,j} \right) }{\partial s^{(p)}_{l_0,j_0}} \right)$$

Az összetett függvény deriválási szabája szerint:

$$\frac{1}{m} \sum_{l=1}^m \sum_{j=1}^c \left( - y_{l,j} \frac{\partial ln \left(\hat y_{l,j} \right) }{\partial s^{(p)}_{l_0,j_0}} \right) = \frac{1}{m} \sum_{l=1}^m \sum_{j=1}^c \left( - y_{l,j} \ \frac{1}{\hat y_{l,j}} \frac{\partial \hat y_{l,j}}{\partial s^{(p)}_{l_0,j_0}} \right)$$

Amennyiben $l \neq l_0$, úgy $\frac{\partial \hat y_{l,j}}{\partial s^{(p)}_{l_0,j_0}} = 0$, azaz egy bemeneti adatpont eredményei függetlenek a többi adatpont eredményeitől:

$$ \frac{1}{m} \sum_{l=1}^m \sum_{j=1}^c \left( - y_{l,j} \ \frac{1}{\hat y_{l,j}} \frac{\partial \hat y_{l,j}}{\partial s^{(p)}_{l_0,j_0}} \right) = \frac{1}{m} \sum_{j=1}^c \left( - y_{l_0,j} \ \frac{1}{\hat y_{l_0,j}} \frac{\partial \hat y_{l_0,j}}{\partial s^{(p)}_{l_0,j_0}} \right)$$ 

A single-label classification esetén (egy kimenet csak egy kategóriához tartozhat) az utolsó réteg aktivációs függvénye a *softmax*, így

$$ \hat y_{l_0,j} = softmax(s^{(p)}_{l_0,j}) = \frac{e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} $$

A törtfüggvény deriválásának szabálya $\left(\frac{f(x)}{g(x)}\right)' = \frac{f'(x)g(x)-g'(x)f(x)}{g^2(x)}$ alapján:

$$ \frac{\partial}{\partial z_{j_0}}softmax(z_{j}) = \frac{\partial}{\partial z_{j_0}}\left(\frac{e^{z_{j}}}{\sum_{j'=1}^{c} e^{z_j'}}\right)$$

Az egyes részfüggvények deriváltjai:

$$ \frac{\partial}{\partial z_{j_0}} e^{z_{j}} =
\begin{cases}
    e^{z_{j_0}}, & \text{ha}\ j = j_0 \\
    0, & \text{ha}\ j \neq j_0
\end{cases}e^{z_{j_0}} $$

$$ \frac{\partial}{\partial z_{j_0}} \sum_{j'=1}^{c} e^{z'_j} = \sum_{j=1}^{c} \frac{\partial}{\partial z_{j_0}}e^{z'_{j}} = e^{z_{j_0}}$$

Ezekből

$$ \frac{\partial \hat y_{l_0,j}}{\partial s^{(p)}_{l_0,j_0}} = \frac{\partial}{\partial s^{(p)}_{l_0,j_0}}softmax(s^{(p)}_{l_0,j}) = \frac{\partial}{\partial s^{(p)}_{l_0,j_0}}\left(\frac{e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}}\right) $$

Ha $j = j_0$:

$$ \frac{\partial}{\partial s^{(p)}_{l_0,j_0}}\left(\frac{e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}}\right) = \frac{e^{s^{(p)}_{l_0,j_0}} \ \sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}} - e^{s^{(p)}_{l_0,j_0}} \ e^{s^{(p)}_{l_0,j}}}{\left( \sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}} \right)^2} = \\
= \frac{e^{s^{(p)}_{l_0,j_0}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} \frac{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}} - e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} = \frac{e^{s^{(p)}_{l_0,j_0}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} \left(\frac{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} - \frac{ e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}}\right) = $$
$$ \\[20pt] $$
$$ = \hat y_{l_0,j_0} \ \left(1 -\hat y_{l_0,j}\right) = \hat y_{l_0,j_0} - \hat y_{l_0,j_0} \ \hat y_{l_0,j}$$

Ha $j \neq j_0$:

$$ \frac{\partial}{\partial s^{(p)}_{l_0,j_0}}\left(\frac{e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}}\right) = \frac{- e^{s^{(p)}_{l_0,j_0}} \ e^{s^{(p)}_{l_0,j}}}{\left( \sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}} \right)^2} = \frac{e^{s^{(p)}_{l_0,j_0}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} \frac{- e^{s^{(p)}_{l_0,j}}}{\sum_{j'=1}^{c} e^{s^{(p)}_{l_0,j'}}} = - \hat y_{l_0,j_0} \ \hat y_{l_0,j}$$

**Megjegyzés:** Az így kiszámított értékek gyakorlatilag az $ \hat y_{l_0,j} = softmax(s^{(p)}_{l_0,j})$ vektor-vektor függvény Jakobi mátrixának elemei:

$$\mathbf{J}\left(\hat y_{l_0,j}\right) = \left[
	\begin{array}{cccc}
 		\frac{\partial \hat y_{l_0,1}}{\partial s^{(p)}_{l_0,1}} & \frac{\partial \hat y_{l_0,1}}{\partial s^{(p)}_{l_0,2}} & \ldots & \frac{\partial \hat y_{l_0,1}}{\partial s^{(p)}_{l_0,c}}\\
		\frac{\partial \hat y_{l_0,2}}{\partial s^{(p)}_{l_0,1}} & \frac{\partial \hat y_{l_0,2}}{\partial s^{(p)}_{l_0,2}} & \ldots & \frac{\partial \hat y_{l_0,2}}{\partial s^{(p)}_{l_0,c}}\\
 		\vdots & \vdots & \ddots & \vdots\\
        \frac{\partial \hat y_{l_0,c}}{\partial s^{(p)}_{l_0,1}} & \frac{\partial \hat y_{l_0,c}}{\partial s^{(p)}_{l_0,2}} & \ldots & \frac{\partial \hat y_{l_0,c}}{\partial s^{(p)}_{l_0,c}}\\
	\end{array}	\right] = $$
$$ \\[20pt] $$
$$    = \left[
	\begin{array}{cccc}
 		\hat y_{l_0,1} - \hat y_{l_0,1} \ \hat y_{l_0,1} &- \hat y_{l_0,2} \ \hat y_{l_0,1} & \ldots & - \hat y_{l_0,c} \ \hat y_{l_0,1}\\
		- \hat y_{l_0,1} \ \hat y_{l_0,2} & \hat y_{l_0,2} - \hat y_{l_0,2} \ \hat y_{l_0,2} & \ldots & - \hat y_{l_0,c} \ \hat y_{l_0,2}\\
 		\vdots & \vdots & \ddots & \vdots\\
        - \hat y_{l_0,1} \ \hat y_{l_0,c} & - \hat y_{l_0,2} \ \hat y_{l_0,c} & \ldots & \hat y_{l_0,c} - \hat y_{l_0,c} \ \hat y_{l_0,c}\\
	\end{array}	\right]$$

Az eredeti $\frac{\partial C_{CCE}}{\partial s_{l_0,j_0}}$ érték számításához visszatérve

$$ \frac{\partial C_{CCE}}{\partial s_{l_0,j_0}} = \frac{1}{m} \sum_{j=1}^c \left( - y_{l_0,j} \ \frac{1}{\hat y_{l_0,j}} \frac{\partial \hat y_{l_0,j}}{\partial s^{(p)}_{l_0,j_0}} \right) = \\
= \frac{1}{m} \left(\left( - y_{l_0,j_0} \ \frac{1}{\hat y_{l_0,j_0}} \hat y_{l_0,j_0} \right) - \sum_{j=1}^c \left( - y_{l_0,j} \ \frac{1}{\hat y_{l_0,j}} \hat y_{l_0,j_0} \ \hat y_{l_0,j} \right) \right) = \\
= \frac{1}{m} \left(- y_{l_0,j_0} - \sum_{j=1}^c \left( - y_{l_0,j} \  \hat y_{l_0,j_0} \right) \right) = \frac{1}{m} \left(\sum_{j=1}^c y_{l_0,j} \  \hat y_{l_0,j_0} - y_{l_0,j_0} \right) = \frac{1}{m} \left(\hat y_{l_0,j_0} \sum_{j=1}^c y_{l_0,j} - y_{l_0,j_0} \right)$$

Az $y_{l_0,j}$ értékek közül a one-hot enkódolásnak megfelelően pontosan egy elem értéke 1, a többi elem értéke 0, így $\sum_{j=1}^c y_{l_0,j} = 1$. Ennek megfelelően

$$ \boxed{ \delta^{(p)}_{l,j} = \frac{\partial C_{CCE}}{\partial s_{l,j}} = \frac{1}{m} \left(\hat y_{l,j} - y_{l,j} \right) } $$

Innen könnyen látható, hogy a multiclass classification feladatra, a CCE költségfüggvény és *softmax* kimeneti aktiváció függvény választással a kimeneti rétegre számított $\mathbf{\delta}^{(p)}$ érték mátrixos formában történő számítása megegyezik a a bináris klasszifikációnál (BCE költség és *sigmoid* kimeneti aktiváció) látottal.

$$ \boxed{ \mathbf{\delta}^{(p)} = \frac{1}{m} \left(\mathbf{\hat Y} - \mathbf{Y}\right)}
